In [9]:
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import pandas as pd

In [14]:
sdir=r'../PartA_Splitted'
aug_dir=os.path.join(sdir,'train_Aug')
original_images_dir=os.path.join(sdir, 'train')

if os.path.isdir(aug_dir): # see if aug_dir exists, if so remove it to get a clean slate
    shutil.rmtree(aug_dir)

os.mkdir(aug_dir) # make a new empty aug_dir
filepaths=[]
labels=[]

 

In [15]:
# iterate through original_images and create a dataframe of the form filepaths, labels


for i in ['Abnormal', 'Normal']:
    os.mkdir(os.path.join(aug_dir,i)) # make the class subdirectories in the aug_dir
    classpath=os.path.join(original_images_dir, i) # get the path to the classes (abnormal and normal)
    flist=os.listdir(classpath)# for each class the the list of files in the class    
    
    for f in flist:        
        fpath=os.path.join(classpath, f) # get the path to the file
        filepaths.append(fpath)
        labels.append(i)
    Fseries=pd.Series(filepaths, name='filepaths')
    Lseries=pd.Series(labels, name='labels')
    
df=pd.concat([Fseries, Lseries], axis=1) # create the dataframe
gen=ImageDataGenerator(rescale=1./255,
                        rotation_range=45,
                        width_shift_range=0.2, 
                        height_shift_range=0.2,
                        horizontal_flip=True)
groups=df.groupby('labels') # group by class

In [16]:
for label in df['labels'].unique():  # for every class               
    group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
    sample_count=len(group)   # determine how many samples there are in this class  
    aug_img_count=0
    target_dir=os.path.join(aug_dir, label)  # define where to write the images    
    aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=(224,224),
                                     class_mode=None, batch_size=32,seed=None, save_to_dir=target_dir,
                                     save_prefix='aug', save_format='jpg')
    while aug_img_count<len(group):
        images=next(aug_gen)            
        aug_img_count += len(images)

Found 730 validated image filenames.
Found 450 validated image filenames.
